1JHC,2JHN(REDO W DROPOUT),3JHN

In [1]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt, seaborn as sns, pickle, numba, torch, tqdm, random, utils, os, gc, time

from collections import OrderedDict, defaultdict
from copy import deepcopy

from rdkit.Chem.AtomPairs.Utils import NumPiElectrons
from rdkit.Chem.rdMolTransforms import GetAngleRad, GetBondLength
from rdkit.Chem.rdchem import Atom, Bond
from rdkit.Chem.AtomPairs.Pairs import rdMolDescriptors as rdmd
from rdkit import Chem, RDConfig
from rdkit.Chem import ChemicalFeatures

import torch_geometric.transforms as T
import torch.nn.functional as F
from torch.nn import Sequential, Linear, ReLU, GRU, BatchNorm1d, Dropout, SELU
from torch_geometric.data import Data
from torch_geometric.nn import NNConv, Set2Set, GATConv
from torch_geometric.data import DataLoader


%config InlineBackend.figure_format ='retina'

In [2]:
structures = pd.read_csv("structures.csv")
test = pd.read_csv('test.csv')
pr = pd.read_csv("1JHNTPreds.csv")

In [3]:
gratio = pd.DataFrame({"atom":["H","C","N","O","F"],
                       "Gratio": [42.576,10.705,-4.316,-5.772,40.060],
                       "Eneg": [2.2,2.55,3.04,3.44,3.98],
                       "radius":[110,170,155,152,147],
                       "atomic_num":[1,6,7,8,9],
                       "numv":[1,4,5,6,7]})
structures = structures.merge(gratio,how='left',on='atom')

In [4]:
test = test.groupby('type').get_group('1JHN')

In [5]:
t2 = pd.read_csv('test.csv')
t2 = t2.groupby('type').get_group('1JHN')
t2['pp'] = pr['scalar_coupling_constant'].values
t2.drop(['id','type'],axis=1,inplace=True)

In [6]:
t2.head()

,molecule_name,atom_index_0,atom_index_1,pp
56,dsgdb9nsd_000020,4,0,48.322174
60,dsgdb9nsd_000020,5,0,52.308125
65,dsgdb9nsd_000020,6,2,48.252357
69,dsgdb9nsd_000020,7,2,52.161621
156,dsgdb9nsd_000036,7,1,55.157257


In [7]:
from collections import defaultdict
molcouples = defaultdict(list)
cpv = t2.values.tolist()
for c in tqdm.tqdm_notebook(cpv):
    molcouples[c[0]].append((c[1],c[2],c[3]))

In [8]:
test.drop('type',axis=1,inplace=True)

In [9]:
test = utils.map_atom_info(test,0,structures)
test = utils.map_atom_info(test,1,structures)

In [10]:
with open('rdkitmolecules.p', 'rb') as fp:
    d = pickle.load(fp)

In [11]:
molnames = list(test['molecule_name'].unique())
mols = OrderedDict()
for name in molnames:
    mols[name] = d[name]
struct =  structures[structures['molecule_name'].isin(molnames)]
g = struct.groupby('molecule_name')
struct.head()

,molecule_name,atom_index,atom,x,y,z,Gratio,Eneg,radius,atomic_num,numv
123,dsgdb9nsd_000020,0,N,0.036053,1.360779,-0.124164,-4.316,3.04,155,7,5
124,dsgdb9nsd_000020,1,C,-0.025911,-0.020766,0.002006,10.705,2.55,170,6,4
125,dsgdb9nsd_000020,2,N,1.219685,-0.623342,0.119632,-4.316,3.04,155,7,5
126,dsgdb9nsd_000020,3,O,-1.068229,-0.641746,0.008656,-5.772,3.44,152,8,6
127,dsgdb9nsd_000020,4,H,0.807494,1.834551,0.321449,42.576,2.20,110,1,1


In [12]:
from rdkit import Chem
from rdkit.Chem import ChemicalFeatures
from rdkit import RDConfig
import os
fdefName = os.path.join(RDConfig.RDDataDir,'BaseFeatures.fdef')
factory = ChemicalFeatures.BuildFeatureFactory(fdefName)

In [13]:
atfn = {'Acceptor':0,
         'Aromatic':1,
         'Donor':2,
         'Hydrophobe':3,
         'LumpedHydrophobe':4,
         'NegIonizable':5,
         'PosIonizable':6,
         'ZnBinder':7}

In [14]:
#ADD DISTANCE TO TARGET FEATURE
def getAtomNodeFeats(molname, mols, data):
    symbol, x, y, z, gr,eneg,radius,atomnum,nv = 2,3,4,5,6,7,8,9,10
    mol = mols[molname]
    feats = factory.GetFeaturesForMol(mol)
    atfdict = defaultdict(list)
    for i in range(len(feats)):
        fam = feats[i].GetFamily()
        dl = feats[i].GetAtomIds()
        for aid in dl:
            atfdict[aid].append(fam)
    
    l = len(data)
    nodef = []
    
    eems = rdmd.CalcEEMcharges(mol)
    spfd = {'S':0,'SP':0.5,'SP2':1/3,'SP3':1/4}
    
    for i in range(l):
        d = data[i]
        atom = mol.GetAtomWithIdx(i)
        ahyb =  str(atom.GetHybridization())
        sym = d[symbol]
        hf = 'NONE'
        if i in atfdict:
            hf = atfdict[i]
        ff = [sym=='H',sym=='C',sym=='N',sym=='O',sym=='F',d[atomnum],(int)(atom.IsInRing() == True),
              ahyb=='S',ahyb=='SP', ahyb=='SP2', ahyb=='SP3',d[x],d[y],d[z],eems[i],spfd[ahyb],
              d[radius],d[eneg],d[gr],NumPiElectrons(atom),0,0]
        ff += ([0]*135)#         
        newf = [0,0,0,0,0,0,0,0, 0,0,0,0, 0,0,0,0,0,0,0,0,0,0]
        newf[-11] = d[nv]
        if hf != 'NONE':
            for t in hf:
                newf[atfn[t]] = 1
            
        ff += newf
        nodef.append(ff)
    
    lm = molcouples[molname]
    for i,(a0,a1,v) in enumerate(lm):
        nodef[a0][22+i] = 1
        nodef[a1][22+i] = 1
    return nodef

In [15]:
@numba.jit(nopython=True)
def getAngle(x0,y0,z0,x1,y1,z1):
    mag_x0 = np.sqrt(x0**2 + y0**2 + z0**2)
    mag_x1 = np.sqrt(x1**2 + y1**2 + z1**2)
    dotp = (x0*x1) + (y0* y1) + (z0*z1)
    c = dotp/(mag_x0*mag_x1)
    if c < -1:
        c = -1
    elif c > 1:
        c = 1
    theta = np.arccos(c)
    return theta, np.cos(theta), np.sin(theta)

In [16]:
def getBondEdgeFeats(molname, mols, data):
    x, y, z = 3,4,5
    mol = mols[molname]
    bonds = mol.GetBonds()
    src,dst = [],[]
    bondf = []
    aed = defaultdict(list)
    
    btd = {'AROMATIC':0,'SINGLE':1,'DOUBLE':2, 'TRIPLE':3}
    #make bidirectional
    for bond in bonds:
        id0,id1 = bond.GetBeginAtomIdx(), bond.GetEndAtomIdx()
        
        src.append(id0)
        dst.append(id1)
        src.append(id1)
        dst.append(id0)
        
        aed[id0].append(id1)
        aed[id1].append(id0)
        
        t0,t1 = data[id0],data[id1]
        x0,y0,z0,x1,y1,z1 = t0[x],t0[y],t0[z],t1[x],t1[y],t1[z]   
                
        d = GetBondLength(mol.GetConformer(),id0,id1)
        a,cos,sin = getAngle(x0,y0,z0,x1,y1,z1)
        isconj = (int)(bond.GetIsConjugated() == True)
        btype =  str(bond.GetBondType())
        bt = [0,0,0,0]
        bt[btd[btype]] = 1
        #angle t0, cos t0, sin t0, angle t1, cos t1, sin t1, angle tv, cos tv, sin tv
        ff = [d,a,cos,sin,isconj] + bt + [0,0,0,0,0,0,0,0,0,0]
        bondf.append(ff)
        bondf.append(ff)
        #[blen,angle,cos,sin,isconj,bt,bt,bt,bt]
    edge_index = []
    edge_index.append(src)
    edge_index.append(dst)
    #tonorm: 0,1,2,3,4
    return bondf, edge_index, aed    

In [17]:
mnames = list(mols.keys())

datadir = {}
for name in tqdm.tqdm_notebook(mnames):
    a = g.get_group(name)
    nodef = getAtomNodeFeats(name,mols,a.values.tolist())
    bondf, edgeidx, aed = getBondEdgeFeats(name,mols,a.values.tolist())
    datadir[name] = [nodef, bondf, edgeidx, aed]

In [18]:
test= test.drop(['atom_0', 'x_0', 'y_0', 'z_0', 'Gratio_x',
            'Eneg_x', 'radius_x', 'atomic_num_x', 'atom_1', 'x_1', 'y_1', 'z_1',
            'Gratio_y', 'Eneg_y', 'radius_y', 'atomic_num_y','numv_x','numv_y'],axis=1)

In [19]:
t = test.values.tolist()

In [20]:
@numba.jit(nopython=True)
def getDist(x0,y0,z0,x1,y1,z1):
    return np.sqrt((x0-x1)**2 + (y0-y1)**2 + (z0-z1)**2)

In [21]:
olds = np.seterr(all='raise')

In [22]:
testlist = []
l = len(t)
ids = []
for dp in tqdm.tqdm_notebook(t):
    name, id0, id1 = dp[1], dp[2], dp[3]
    lm2 = molcouples[name]
    n,b,e,aed = datadir[name]
    nodef,bondf,edgeidx = np.array(n), np.array(b), np.array(e)
    nodef,bondf,edgeidx = nodef.tolist(),bondf.tolist(),edgeidx.tolist()
    
    nodef[id0][-1] = 1
    nodef[id1][-1] = 1
    
    nid0 = nodef[id0]
    nid1 = nodef[id1]
    x0,y0,z0 = nid0[11],nid0[12],nid0[13]
    x1,y1,z1 = nid1[11],nid1[12],nid1[13]
    xf,yf,zf = x0-x1,y0-y1,z0-z1
    
    ll = len(nodef)
    ai0 = aed[id0]
    ai1 = aed[id1]
    for i in range(ll):
        if i not in ai0 and i != id0:
            temp = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
            tnf = nodef[i]
            xi,yi,zi = tnf[11],tnf[12],tnf[13]
            temp[0] = getDist(xi,yi,zi,x0,y0,z0)
            theta,cos,sin = getAngle(xi,yi,zi,x0,y0,z0)
            temp[1] = theta
            temp[2] = cos
            temp[3] = sin
            
            bondf.append(temp)
            bondf.append(temp)
            
            edgeidx[0].append(i)
            edgeidx[1].append(id0)
            
            edgeidx[0].append(id0)
            edgeidx[1].append(i)
        if i not in ai1 and  i != id1:
            temp = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
            tnf = nodef[i]
            xi,yi,zi = tnf[11],tnf[12],tnf[13]
            temp[0] = getDist(xi,yi,zi,x1,y1,z1)
            theta,cos,sin = getAngle(xi,yi,zi,x1,y1,z1)
            temp[1] = theta
            temp[2] = cos
            temp[3] = sin
            
            bondf.append(temp)
            bondf.append(temp)
            
            edgeidx[0].append(i)
            edgeidx[1].append(id1)
            
            edgeidx[0].append(id1)
            edgeidx[1].append(i)
    bonds = [x for x in zip(edgeidx[0],edgeidx[1])]
    for i,bf in enumerate(bondf):
        i0,i1 = bonds[i]
        for i,(a0,a1,v) in enumerate(lm2):
            if (a0,a1) == (i0,i1):
                bf[-10] = v
        nfa,nfb = nodef[i0],nodef[i1]
        xa,ya,za = nfa[11],nfa[12],nfa[13]
        xb,yb,zb = nfb[11],nfb[12],nfb[13]
        xi,yi,zi = xa-xb,ya-yb,za-zb
        a0,cos0,sin0 = getAngle(xi,yi,zi,x0,y0,z0)
        a1,cos1,sin1 = getAngle(xi,yi,zi,x1,y1,z1)
        a2,cos2,sin2 = getAngle(xi,yi,zi,xf,yf,zf)

        bf[-9] = a0
        bf[-8] = cos0
        bf[-7] = sin0
        bf[-6] = a1
        bf[-5] = cos1
        bf[-4] = sin1
        bf[-3] = a2
        bf[-2] = cos2
        bf[-1] = sin2

    for i,nf in enumerate(nodef):
        if i == id0:
            nf[20] = 0
            nf[21] = getDist(x0,y0,z0,x1,y1,z1)
            ang,cos,sin = getAngle(x0,y0,z0,xf,yf,zf)
            nf[-14] = ang
            nf[-13] = cos
            nf[-12] = sin
            nf[-7] = 0
            nf[-6] = 1
            nf[-5] = 0
            a1,cos,sin = getAngle(x0,y0,z0,x1,y1,z1)
            nf[-4] = a1
            nf[-3] = cos
            nf[-2] = sin
            nf[-10] = x1-x0
            nf[-9] = y1-y0
            nf[-8] = z1-z0
            nf[11] = 0
            nf[12] = 0
            nf[13] = 0
            
        elif i == id1:
            nf[21] = 0
            nf[20] = getDist(x0,y0,z0,x1,y1,z1)
            ang,cos,sin = getAngle(x1,y1,z1,xf,yf,zf)
            nf[-14] = ang
            nf[-13] = cos
            nf[-12] = sin
            a0,cos,sin = getAngle(x0,y0,z0,x1,y1,z1)
            nf[-7] = a0
            nf[-6] = cos
            nf[-5] = sin
            nf[-4] = 0
            nf[-3] = 1
            nf[-2] = 0
            nf[-10] = 0
            nf[-9] = 0
            nf[-8] = 0
            nf[11] = x0-x1
            nf[12] = y0-y1
            nf[13] = z0-z1
        else:
            xi,yi,zi = nf[11],nf[12],nf[13]
            nf[21] = getDist(xi,yi,zi,x1,y1,z1)
            nf[20] = getDist(xi,yi,zi,x0,y0,z0)
            ang,cos,sin = getAngle(xi,yi,zi,xf,yf,zf)
            nf[-14] = ang
            nf[-13] = cos
            nf[-12] = sin
            a0,cos0,sin0 = getAngle(xi,yi,zi,x0,y0,z0)
            a1,cos1,sin1 = getAngle(xi,yi,zi,x1,y1,z1)
            nf[-7] = a0
            nf[-6] = cos0
            nf[-5] = sin0
            nf[-4] = a1
            nf[-3] = cos1
            nf[-2] = sin1
            nf[-10] = x1-xi
            nf[-9] = y1-yi
            nf[-8] = z1-zi
            nf[11] = x0-xi
            nf[12] = y0-yi
            nf[13] = z0-zi
    
    ids.append(dp[0])
    x = torch.tensor(nodef, dtype=torch.float)
    edge_index = torch.tensor(edgeidx, dtype=torch.long)
    edge_attr = torch.tensor(bondf,dtype=torch.float)
    data = Data(x=x, edge_index=edge_index, edge_attr=edge_attr)
    
    testlist.append(data)

In [23]:
tmean = 47.47988448446838
tsd = 10.922171556272295

In [24]:
test_loader = DataLoader(testlist, batch_size=256,shuffle=False)

In [25]:
ndim =128
edim = 64

In [26]:
torch.cuda.empty_cache()

In [27]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.lin0 = torch.nn.Linear(179, ndim)
        self.bn0 = BatchNorm1d(ndim)
        self.d0 = Dropout(0.1)
        
        nn = Sequential(Linear(20, edim), ReLU(), Linear(edim,2*edim), ReLU(),Linear(2*edim, ndim * ndim))
        self.conv = NNConv(ndim, ndim, nn, aggr='max', root_weight=True)
        self.gru = GRU(ndim, ndim)
        
        self.set2set = Set2Set(ndim, processing_steps=3)
        self.lin1 = torch.nn.Linear(2*ndim, 192)
        self.lin2 = torch.nn.Linear(192,160)
        self.lin3 = torch.nn.Linear(160, ndim)
        self.lin4 = torch.nn.Linear(ndim, 1)

    def forward(self, data):
        out = F.selu(self.lin0(data.x))
        out = self.bn0(out)
        out = self.d0(out)
        h = out.unsqueeze(0)
        
        for i in range(3):
            m = F.selu(self.conv(out, data.edge_index, data.edge_attr))
            out, h = self.gru(m.unsqueeze(0), h)
            out = out.squeeze(0)
        
        out = self.set2set(out, data.batch)
        out = F.selu(self.lin1(out))
        out = F.selu(self.lin2(out))
        out = F.selu(self.lin3(out))
        out = self.lin4(out)
        
        return out.view(-1)

In [28]:
model2 = torch.load('MPNN_1JHN_L4D.pth')
#model2 = torch.load('MPNN_2JHN_SC2T_NFR.pth')
#xd = torch.load('Temp/t1.pth')

In [29]:
#model2 = xd['bestmodel']

In [30]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model2.to(device)
model2.eval()
preds = []
for dd in tqdm.tqdm_notebook(test_loader):
    dd = dd.to(device)
    x = model2(dd)
    x = x.cpu().detach().numpy()
    x = (x*tsd)+tmean
    x = list(x)
    preds +=x


In [31]:
print(len(preds),len(ids))

24195 24195


In [33]:
s = pd.read_csv("77_MPNN_NFR_1_2_3JHC2.csv")

In [34]:
s =  s[~s['id'].isin(ids)]

In [35]:
s.head()

,id,scalar_coupling_constant
0,4658205,-0.221141
1,4658209,1.953058
2,4658210,-0.222225
3,4658214,1.948133
4,4658555,-0.560792


In [36]:
new = pd.DataFrame({'id':ids, 'scalar_coupling_constant':preds})

In [37]:
ff = pd.concat([new,s])

In [38]:
len(ff)

2505542

In [39]:
ff.to_csv("78_1JHNR.csv",index=False)

In [ ]:
len(ff)

In [ ]:
ff.head()

In [ ]:
s = pd.read_csv("C:/Users/Alamjeet Singh/AA_SCCKaggle/Restart/26_lgbm_RDKCCType.csv")

In [ ]:
len(s)

Ensemble

In [ ]:
import pandas as pd, numpy as np

In [ ]:
lgb = pd.read_csv("C:/Users/Alamjeet Singh/AA_SCCKaggle/Restart/26_lgbm_RDKCCType.csv")
nn = pd.read_csv("C:/Users/Alamjeet Singh/AA_SCCKaggle/AA/27_MPNN1JHC_LGBM.csv")

In [ ]:
train = pd.read_csv("C:/Users/Alamjeet Singh/Downloads/champs-scalar-coupling/test.csv")
train = train.groupby('type').get_group('1JHC')

In [ ]:
train.head()

In [ ]:
ids = train['id'].tolist()

In [ ]:
nn = nn[nn['id'].isin(ids)]
lgb = lgb[lgb['id'].isin(ids)]

In [ ]:
nn.sort_values(by=['id'],inplace=True)
lgb.sort_values(by=['id'],inplace=True)

In [ ]:
nn.reset_index(inplace=True)
lgb.reset_index(inplace=True)

In [ ]:
ens = (nn['scalar_coupling_constant'] + lgb['scalar_coupling_constant'])/2

In [ ]:
ff = pd.DataFrame({'id':nn['id'],'scalar_coupling_constant':ens})

In [ ]:
len(ff)

In [ ]:
n = pd.read_csv("C:/Users/Alamjeet Singh/AA_SCCKaggle/AA/27_MPNN1JHC_LGBM.csv")

In [ ]:
len(n)

In [ ]:
n = n[~n['id'].isin(ff['id'])]

In [ ]:
len(n)

In [ ]:
ff.head()

In [ ]:
n.head()

In [ ]:
d = pd.concat([ff,n])

In [ ]:
len(d)

In [ ]:
d.to_csv("28_MPNN1JHC_LGBM_ENS.csv",index=False)

In [ ]:
#add GAT layer
#try lin first, then conv3 in for loop only
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.lin0 = torch.nn.Linear(166, ndim)
        self.bn0 = BatchNorm1d(ndim)
        self.d0 = torch.nn.Dropout(0.1)
        
        nn = Sequential(Linear(9, edim), ReLU(), Linear(edim, ndim * ndim))
        self.conv = NNConv(ndim, ndim, nn, aggr='max', root_weight=True)
        self.gru = GRU(ndim, ndim)

        self.set2set = Set2Set(ndim, processing_steps=3)
        self.lin1 = torch.nn.Linear(2*ndim, ndim)
        #self.d1 = torch.nn.Dropout(0.5)
        self.lin2 = torch.nn.Linear(ndim, 1)

    def forward(self, data):
        out = F.relu(self.lin0(data.x))
        out = self.bn0(out)
        out = self.d0(out)
        h = out.unsqueeze(0)
        
        for i in range(3):
            m = F.relu(self.conv(out, data.edge_index, data.edge_attr))
            out, h = self.gru(m.unsqueeze(0), h)
            out = out.squeeze(0)
        
        out = self.set2set(out, data.batch)
        out = F.relu(self.lin1(out))
        #out = self.d1(out)
        out = self.lin2(out)
        
        return out.view(-1)